# 🛡️ Siamese Network for Signature Verification

**Training a signature verification model using Siamese Networks with MobileNetV2 backbone**

- Dataset: CEDAR + BHSig260 (Hindi & Bengali)
- Model: Siamese Network with shared weights
- Loss: Binary Cross Entropy
- GPU: Kaggle P100/T4

In [ ]:
# Cell 1: Install Dependencies (if needed)
!pip install torch torchvision tqdm matplotlib pillow scikit-learn --quiet
print("✅ Dependencies installed!")

In [ ]:
# Cell 2: Imports and Configuration
import os
import random
import time
from itertools import combinations
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Cell 3: Configuration
class Config:
    # Dataset paths (Kaggle)
    DATASET_BASE = '/kaggle/input/handwritten-signature-datasets'
    CEDAR_PATH = os.path.join(DATASET_BASE, 'signatures/signatures')
    # BHSig paths - adjust based on actual structure
    BHSIG_HINDI = os.path.join(DATASET_BASE, 'BHSig260/Hindi')
    BHSIG_BENGALI = os.path.join(DATASET_BASE, 'BHSig260/Bengali')
    
    # Image settings
    IMAGE_SIZE = (155, 220)
    GRAYSCALE = True
    
    # Model settings
    EMBEDDING_DIM = 512
    BACKBONE = 'mobilenet_v2'
    
    # Training settings
    BATCH_SIZE = 64  # Increased for GPU
    LEARNING_RATE = 0.0001
    NUM_EPOCHS = 50
    TRAIN_SPLIT = 0.8
    NUM_WORKERS = 4  # Multi-process data loading
    
    # Checkpoints
    CHECKPOINT_INTERVAL = 10
    EARLY_STOPPING_PATIENCE = 10
    SAVE_PATH = '/kaggle/working/checkpoints'
    
    RANDOM_SEED = 42

config = Config()
os.makedirs(config.SAVE_PATH, exist_ok=True)
print("✅ Configuration loaded!")

In [ ]:
# Cell 4: Explore Dataset Structure
print("📁 Dataset Structure:")
print("="*50)

# List top-level
if os.path.exists(config.DATASET_BASE):
    for item in os.listdir(config.DATASET_BASE):
        path = os.path.join(config.DATASET_BASE, item)
        if os.path.isdir(path):
            count = len(os.listdir(path))
            print(f"📂 {item}/ ({count} items)")
            # Show first few items
            for sub in os.listdir(path)[:3]:
                print(f"   └── {sub}")
        else:
            print(f"📄 {item}")
else:
    print("❌ Dataset not found! Make sure to add the dataset to your notebook.")

In [ ]:
# Cell 5: Dataset Classes

def get_file_type(filename, dataset_type):
    """Determine if file is genuine or forged"""
    filename_lower = filename.lower()
    
    if dataset_type == 'cedar':
        if 'original' in filename_lower:
            return 'genuine'
        elif 'forgeries' in filename_lower:
            return 'forged'
    else:  # bhsig
        if '-g-' in filename_lower:
            return 'genuine'
        elif '-f-' in filename_lower:
            return 'forged'
    return None


def load_dataset_images(dataset_path, dataset_type):
    """Load images organized by person"""
    data = {}
    
    if not os.path.exists(dataset_path):
        print(f"Warning: {dataset_path} not found")
        return data
    
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                file_path = os.path.join(root, file)
                file_type = get_file_type(file, dataset_type)
                
                if file_type is None:
                    continue
                
                # Extract person ID from path
                rel_path = os.path.relpath(root, dataset_path)
                if rel_path == '.':
                    continue
                person_id = f"{dataset_type}_{rel_path.split(os.sep)[0]}"
                
                if person_id not in data:
                    data[person_id] = {'genuine': [], 'forged': []}
                
                data[person_id][file_type].append(file_path)
    
    return data


def generate_balanced_pairs(data, max_pairs_per_person=50):
    """Generate balanced genuine and forgery pairs"""
    genuine_pairs = []
    forgery_pairs = []
    
    for person_id, images in data.items():
        genuine_images = images['genuine']
        forged_images = images['forged']
        
        if len(genuine_images) < 2:
            continue
        
        # Genuine pairs
        person_genuine = list(combinations(genuine_images, 2))
        random.shuffle(person_genuine)
        person_genuine = person_genuine[:max_pairs_per_person]
        
        # Forgery pairs
        person_forgery = [(g, f) for g in genuine_images for f in forged_images]
        random.shuffle(person_forgery)
        person_forgery = person_forgery[:max_pairs_per_person]
        
        genuine_pairs.extend(person_genuine)
        forgery_pairs.extend(person_forgery)
    
    # Balance
    min_pairs = min(len(genuine_pairs), len(forgery_pairs))
    random.shuffle(genuine_pairs)
    random.shuffle(forgery_pairs)
    genuine_pairs = genuine_pairs[:min_pairs]
    forgery_pairs = forgery_pairs[:min_pairs]
    
    # Combine
    pairs = genuine_pairs + forgery_pairs
    labels = [1] * len(genuine_pairs) + [0] * len(forgery_pairs)
    
    combined = list(zip(pairs, labels))
    random.shuffle(combined)
    pairs, labels = zip(*combined)
    
    return list(pairs), list(labels)


class SignatureDataset(Dataset):
    def __init__(self, pairs, labels, transform=None):
        self.pairs = pairs
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        img1_path, img2_path = self.pairs[idx]
        label = self.labels[idx]
        
        img1 = Image.open(img1_path).convert('L' if config.GRAYSCALE else 'RGB')
        img2 = Image.open(img2_path).convert('L' if config.GRAYSCALE else 'RGB')
        
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        
        return img1, img2, torch.tensor(label, dtype=torch.float32)

print("✅ Dataset classes defined!")

In [ ]:
# Cell 6: Load and Prepare Data
random.seed(config.RANDOM_SEED)

print("📦 Loading datasets...")

all_data = {}

# Load CEDAR
if os.path.exists(config.CEDAR_PATH):
    cedar_data = load_dataset_images(config.CEDAR_PATH, 'cedar')
    all_data.update(cedar_data)
    print(f"  CEDAR: {len(cedar_data)} persons")

# Load BHSig Hindi
if os.path.exists(config.BHSIG_HINDI):
    hindi_data = load_dataset_images(config.BHSIG_HINDI, 'bhsig')
    all_data.update(hindi_data)
    print(f"  BHSig Hindi: {len(hindi_data)} persons")

# Load BHSig Bengali  
if os.path.exists(config.BHSIG_BENGALI):
    bengali_data = load_dataset_images(config.BHSIG_BENGALI, 'bhsig')
    all_data.update(bengali_data)
    print(f"  BHSig Bengali: {len(bengali_data)} persons")

print(f"\nTotal persons: {len(all_data)}")

# Generate pairs
pairs, labels = generate_balanced_pairs(all_data, max_pairs_per_person=100)
print(f"Total balanced pairs: {len(pairs)}")
print(f"Genuine pairs: {sum(labels)}")
print(f"Forgery pairs: {len(labels) - sum(labels)}")

# Split
split_idx = int(len(pairs) * config.TRAIN_SPLIT)
train_pairs, train_labels = pairs[:split_idx], labels[:split_idx]
val_pairs, val_labels = pairs[split_idx:], labels[split_idx:]

print(f"\nTraining pairs: {len(train_pairs)}")
print(f"Validation pairs: {len(val_pairs)}")

In [ ]:
# Cell 7: Create DataLoaders

# Transforms
train_transform = transforms.Compose([
    transforms.Resize(config.IMAGE_SIZE),
    transforms.RandomRotation(5),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

val_transform = transforms.Compose([
    transforms.Resize(config.IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Datasets
train_dataset = SignatureDataset(train_pairs, train_labels, train_transform)
val_dataset = SignatureDataset(val_pairs, val_labels, val_transform)

# DataLoaders
train_loader = DataLoader(
    train_dataset, batch_size=config.BATCH_SIZE, shuffle=True,
    num_workers=config.NUM_WORKERS, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=config.BATCH_SIZE, shuffle=False,
    num_workers=config.NUM_WORKERS, pin_memory=True
)

print(f"✅ DataLoaders ready!")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")

In [ ]:
# Cell 8: Siamese Network Model

class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim=512):
        super(SiameseNetwork, self).__init__()
        
        # MobileNetV2 backbone
        self.backbone = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        
        # Modify for grayscale
        if config.GRAYSCALE:
            self.backbone.features[0][0] = nn.Conv2d(
                1, 32, kernel_size=3, stride=2, padding=1, bias=False
            )
        
        self.backbone.classifier = nn.Identity()
        
        # Embedding layers
        self.embedding = nn.Sequential(
            nn.Linear(1280, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, embedding_dim),
            nn.ReLU(inplace=True)
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward_one(self, x):
        features = self.backbone(x)
        if len(features.shape) > 2:
            features = F.adaptive_avg_pool2d(features, 1)
            features = features.view(features.size(0), -1)
        return self.embedding(features)
    
    def forward(self, img1, img2):
        emb1 = self.forward_one(img1)
        emb2 = self.forward_one(img2)
        diff = torch.abs(emb1 - emb2)
        return self.classifier(diff).squeeze()

# Create model
model = SiameseNetwork(config.EMBEDDING_DIM).to(device)

# Print summary
total_params = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✅ Model created!")
print(f"   Total params: {total_params:,}")
print(f"   Trainable: {trainable:,}")

In [ ]:
# Cell 9: Training Functions

def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    
    for img1, img2, labels in tqdm(loader, desc='Training'):
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(img1, img2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    return running_loss / len(loader), 100 * correct / total


def validate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    
    with torch.no_grad():
        for img1, img2, labels in tqdm(loader, desc='Validating'):
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
            outputs = model(img1, img2)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            preds = (outputs > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    return running_loss / len(loader), 100 * correct / total

print("✅ Training functions defined!")

In [ ]:
# Cell 10: Training Loop

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)

history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_val_acc = 0.0
no_improve = 0

print("="*60)
print("🚀 Starting Training")
print("="*60)

start_time = time.time()

for epoch in range(config.NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{config.NUM_EPOCHS}")
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(model, val_loader, criterion)
    
    scheduler.step(val_acc)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        no_improve = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'val_acc': val_acc
        }, os.path.join(config.SAVE_PATH, 'best_model.pth'))
        print(f"  ⭐ Best model saved! Acc: {val_acc:.2f}%")
    else:
        no_improve += 1
        print(f"  ⚠️ No improvement ({no_improve}/{config.EARLY_STOPPING_PATIENCE})")
    
    # Periodic checkpoint
    if (epoch + 1) % config.CHECKPOINT_INTERVAL == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'val_acc': val_acc,
            'history': history
        }, os.path.join(config.SAVE_PATH, f'checkpoint_epoch_{epoch+1}.pth'))
        print(f"  📁 Checkpoint saved: epoch {epoch+1}")
    
    # Early stopping
    if no_improve >= config.EARLY_STOPPING_PATIENCE:
        print(f"\n⛔ Early stopping at epoch {epoch+1}")
        break

total_time = time.time() - start_time
print("\n" + "="*60)
print("🎉 Training Complete!")
print(f"   Time: {total_time/60:.1f} minutes")
print(f"   Best Val Acc: {best_val_acc:.2f}%")
print("="*60)

In [ ]:
# Cell 11: Plot Training History

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train', linewidth=2)
axes[0].plot(history['val_loss'], label='Validation', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train', linewidth=2)
axes[1].plot(history['val_acc'], label='Validation', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training and Validation Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(config.SAVE_PATH, 'training_history.png'), dpi=150)
plt.show()

In [ ]:
# Cell 12: List Saved Models

print("📁 Saved Models:")
print("="*50)
for f in sorted(os.listdir(config.SAVE_PATH)):
    if f.endswith('.pth'):
        path = os.path.join(config.SAVE_PATH, f)
        size = os.path.getsize(path) / (1024*1024)
        print(f"  {f} ({size:.1f} MB)")

print("\n💡 Download the best_model.pth file for deployment!")